In [1]:
var('t q')

def L_partition_vector2(partition, vector):
    for i in range(len(partition)):
        vector = d[-partition[len(partition) - i - 1]]*vector
    return vector

def my_coefficient(partition, vector):
    return vector.monomial_coefficients().get(tuple(-k for k in partition), 0)

def series_t_q(list_of_list_of_partition):
    series = 0
    for i in range(len(list_of_list_of_partition)):
        series_t = 0
        for j in range(len(list_of_list_of_partition[i])):
            series_t = series_t + t^len(list_of_list_of_partition[i][j])
        series = series + series_t*q^i
    return series

def partition_ends(partition, tail, is_greater):
    if len(partition) < len(tail) - 1:
        return False
    partition2 = partition[len(partition) - len(tail) + 1: len(partition)]
    if partition2 != tail[1:]:
        return False
    if is_greater:
        if partition == partition2:
            return True
        if partition[len(partition) - len(tail)] > tail[0]:
            return True
    else:
        if partition == partition2:
            return False
        if partition[len(partition) - len(tail)] == tail[0]:
            return True
    return False

def list_of_list_of_partition_ends(list_of_list_of_partition, tail, is_greater):
    return [[partition
             for partition in list_of_partition if partition_ends(partition, tail, is_greater)]
            for list_of_partition in list_of_list_of_partition]

def my_order(partition):
    polynomial = Partition(partition).to_exp(sum(partition))
    polynomial = [-k for k in polynomial]
    polynomial.reverse()
    return ((len(partition), polynomial)) 

def ordered_partitions(N):
    if N <= 0:
        return [[]]
    list_of_partition = list(Partitions(N))
    list_of_partition.sort(key = my_order, reverse = True)
    return list_of_partition

def find_singular_vector_at_weight(weight):
    list_of_partition = ordered_partitions(weight)
    list_of_partition.reverse()
    list_of_partition2 = ordered_partitions(weight - 1)
    list_of_partition2.reverse() 
    list_of_partition3 = ordered_partitions(weight - 2)
    list_of_partition3.reverse()
    matrix_1 = matrix({(i, j): my_coefficient(list_of_partition2[i], d[1]*L_partition_vector2(list_of_partition[j], v)) for j in range(len(list_of_partition)) for i in range(len(list_of_partition2))}, sparse = False)
    matrix_2 = matrix({(i, j): my_coefficient(list_of_partition3[i], d[2]*L_partition_vector2(list_of_partition[j], v)) for j in range(len(list_of_partition)) for i in range(len(list_of_partition3))}, sparse = False)
    matrix_1 = matrix_1.stack(matrix_2)
    matrix_1 = matrix_1.rref()
    vector = [-matrix_1[k, len(list_of_partition)-1] for k in range(len(list_of_partition)-1)]
    vector_list = list(reversed(vector + [1]))
    denominators = [value.denominator() for value in vector_list]
    common_denominator = LCM(denominators)
    vector_list = [value*common_denominator for value in vector_list]
    list_of_partition.reverse()
    vector2 = {tuple(list_of_partition[i]): vector_list[i] for i in range(len(list_of_partition))}
    return vector2

def find_singular_vectors():
    return (find_singular_vector_at_weight(m*n), find_singular_vector_at_weight((pp - m)*(qq - n)))

def vector_sum(vector1, vector2):
    return {partition: vector1.get(partition, 0) + vector2.get(partition, 0)
            for partition in set(vector1)|set(vector2)}

def vector_simplify0(partition):
    for i in range(len(partition) - 1):
        if partition[i] < partition[i + 1]:
            return {partition[0: i] + (partition[i] + partition[i + 1],)
                    + partition[i + 2: len(partition)]: partition[i + 1] - partition[i],
                    partition[0: i] + (partition[i + 1],) + (partition[i],)
                    + partition[i + 2: len(partition)]: 1}
    return {partition: 1}

def vector_simplify(vector):
    vector2 = {}
    for partition in set(vector):
        vector3 = vector_simplify0(partition)
        vector2 = vector_sum(vector2, {partition2: vector3[partition2]*vector[partition]
                                       for partition2 in set(vector3)})
    if vector2 == vector:
        return vector2
    return vector_simplify(vector2)

def L_partition_vector(partition, vector):
    return vector_simplify(
        {partition + partition2: vector[partition2] for partition2 in set(vector)})

def matrix_UVir_m1_m2(N):
    singular_vectors = find_singular_vectors()
    m1 = singular_vectors[0]
    m2 = singular_vectors[1]
    list_of_partition = ordered_partitions(N)
    list_of_partition2 = ordered_partitions(N - m*n)
    list_of_partition3 = ordered_partitions(N - (pp - m)*(qq - n))
    list_of_vector_m1 = [L_partition_vector(tuple(list_of_partition2[i]), m1)
                         for i in range(len(list_of_partition2))]
    list_of_vector_m2 = [L_partition_vector(tuple(list_of_partition3[i]), m2)
                         for i in range(len(list_of_partition3))]
    matrix_m1 = matrix({(i, j): list_of_vector_m1[i].get(tuple(list_of_partition[j]), 0)
                        for j in range(len(list_of_partition))
                        for i in range(len(list_of_partition2))}, sparse = False)
    matrix_m2 = matrix({(i, j): list_of_vector_m2[i].get(tuple(list_of_partition[j]), 0)
                        for j in range(len(list_of_partition))
                        for i in range(len(list_of_partition3))}, sparse = False)
    matrix_m1 = matrix_m1.stack(matrix_m2)
    matrix_m1 = matrix_m1.rref()
    return matrix_m1

def pivots_up_to(N):
    list_of_list_of_partition = []
    for i in range(N + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [list_of_partition[j] for j in matrix_UVir_m1_m2(i).pivots()])
    return list_of_list_of_partition

def basis_up_to(N):
    list_of_list_of_partition = []
    pivots = pivots_up_to(N)
    for i in range(N + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [partition for partition in list_of_partition if partition not in pivots[i]])
    return list_of_list_of_partition

def is_subpartition(partition, partition2):
    if len(partition) < len(partition2):
        return False
    for i in range(len(partition) - len(partition2) + 1):
        if partition[i: i + len(partition2)] == partition2:
            return True
    return False

def partition_contains_any_of_list(partition, list_of_partition):
    for partition2 in list_of_partition:
        if is_subpartition(partition, partition2):
            return True
    return False

def partition_contains_any_of_R(partition, s, list_of_partition):
    if partition == []:
        return False
    if partition_contains_any_of_list(partition, list_of_partition):
        return True
    if len(partition) < s:
        return False
    for i in range(len(partition) - s + 1):
        if partition[i] - partition[i + s - 1] < 2:
            return True
    return False

def basis_up_tov2(N, s, list_of_partition):
    list_of_list_of_partition = []
    for i in range(N + 1):
        list_of_partition2 = ordered_partitions(i)
        list_of_list_of_partition.append(
            [partition for partition in list_of_partition2
             if not partition_contains_any_of_R(partition, s, list_of_partition)])
    return list_of_list_of_partition

In [2]:
ordered_partitions(6)

[[1, 1, 1, 1, 1, 1],
 [2, 1, 1, 1, 1],
 [2, 2, 1, 1],
 [3, 1, 1, 1],
 [2, 2, 2],
 [3, 2, 1],
 [4, 1, 1],
 [3, 3],
 [4, 2],
 [5, 1],
 [6]]

In [3]:
Vir = lie_algebras.VirasoroAlgebra(QQ)
pp = 2
ss = 4
m = 1
n = 4
NN = 20
qq = 2*ss + 1
c = 1 - 6*(pp - qq)^2/(pp*qq)
h = ((n*pp - m*qq)^2 - (pp - qq)^2)/(4*pp*qq)
M = Vir.verma_module(c, h)
d = Vir.basis()
v = M.highest_weight_vector()

In [4]:
(c, h)

(-46/3, -2/3)

In [5]:
%time basis_up_to(NN) == basis_up_tov2(NN, ss, [[1, 1, 1, 1]])

CPU times: user 28.4 s, sys: 73.9 ms, total: 28.5 s
Wall time: 30.9 s


True

In [6]:
%time w = basis_up_tov2(40, ss, [[1, 1, 1, 1]])

CPU times: user 3.77 s, sys: 92.4 ms, total: 3.87 s
Wall time: 4.19 s


In [7]:
series_t_q(w)

(45*t^10 + 361*t^9 + 1072*t^8 + 1680*t^7 + 1655*t^6 + 1080*t^5 + 477*t^4 + 133*t^3 + 20*t^2 + t)*q^40 + (27*t^10 + 272*t^9 + 865*t^8 + 1433*t^7 + 1456*t^6 + 980*t^5 + 440*t^4 + 127*t^3 + 19*t^2 + t)*q^39 + (17*t^10 + 198*t^9 + 699*t^8 + 1212*t^7 + 1283*t^6 + 885*t^5 + 410*t^4 + 120*t^3 + 19*t^2 + t)*q^38 + (9*t^10 + 144*t^9 + 553*t^8 + 1023*t^7 + 1119*t^6 + 799*t^5 + 377*t^4 + 114*t^3 + 18*t^2 + t)*q^37 + (5*t^10 + 101*t^9 + 439*t^8 + 854*t^7 + 980*t^6 + 717*t^5 + 350*t^4 + 108*t^3 + 18*t^2 + t)*q^36 + (2*t^10 + 70*t^9 + 339*t^8 + 712*t^7 + 846*t^6 + 644*t^5 + 320*t^4 + 102*t^3 + 17*t^2 + t)*q^35 + (t^10 + 46*t^9 + 263*t^8 + 586*t^7 + 734*t^6 + 574*t^5 + 296*t^4 + 96*t^3 + 17*t^2 + t)*q^34 + (31*t^9 + 197*t^8 + 481*t^7 + 628*t^6 + 512*t^5 + 269*t^4 + 91*t^3 + 16*t^2 + t)*q^33 + (18*t^9 + 149*t^8 + 389*t^7 + 539*t^6 + 453*t^5 + 248*t^4 + 85*t^3 + 16*t^2 + t)*q^32 + (11*t^9 + 107*t^8 + 314*t^7 + 455*t^6 + 401*t^5 + 224*t^4 + 80*t^3 + 15*t^2 + t)*q^31 + (6*t^9 + 78*t^8 + 248*t^7 + 387*t^6

In [8]:
series_t_q(list_of_list_of_partition_ends(w, [1], True))

(11*t^9 + 149*t^8 + 481*t^7 + 734*t^6 + 644*t^5 + 350*t^4 + 114*t^3 + 19*t^2 + t)*q^40 + (6*t^9 + 107*t^8 + 389*t^7 + 628*t^6 + 574*t^5 + 320*t^4 + 108*t^3 + 18*t^2 + t)*q^39 + (3*t^9 + 78*t^8 + 314*t^7 + 539*t^6 + 512*t^5 + 296*t^4 + 102*t^3 + 18*t^2 + t)*q^38 + (t^9 + 53*t^8 + 248*t^7 + 455*t^6 + 453*t^5 + 269*t^4 + 96*t^3 + 17*t^2 + t)*q^37 + (t^9 + 37*t^8 + 196*t^7 + 387*t^6 + 401*t^5 + 248*t^4 + 91*t^3 + 17*t^2 + t)*q^36 + (23*t^8 + 151*t^7 + 321*t^6 + 352*t^5 + 224*t^4 + 85*t^3 + 16*t^2 + t)*q^35 + (15*t^8 + 116*t^7 + 269*t^6 + 309*t^5 + 205*t^4 + 80*t^3 + 16*t^2 + t)*q^34 + (8*t^8 + 86*t^7 + 220*t^6 + 268*t^5 + 184*t^4 + 75*t^3 + 15*t^2 + t)*q^33 + (5*t^8 + 64*t^7 + 181*t^6 + 233*t^5 + 168*t^4 + 70*t^3 + 15*t^2 + t)*q^32 + (2*t^8 + 45*t^7 + 144*t^6 + 200*t^5 + 149*t^4 + 65*t^3 + 14*t^2 + t)*q^31 + (t^8 + 32*t^7 + 117*t^6 + 172*t^5 + 135*t^4 + 61*t^3 + 14*t^2 + t)*q^30 + (21*t^7 + 90*t^6 + 145*t^5 + 119*t^4 + 56*t^3 + 13*t^2 + t)*q^29 + (14*t^7 + 71*t^6 + 123*t^5 + 107*t^4 + 52*t

In [9]:
series_t_q(list_of_list_of_partition_ends(w, [1, 1], True))

(t^10 + 63*t^9 + 318*t^8 + 580*t^7 + 561*t^6 + 319*t^5 + 108*t^4 + 18*t^3 + t^2)*q^40 + (43*t^9 + 251*t^8 + 495*t^7 + 499*t^6 + 295*t^5 + 102*t^4 + 18*t^3 + t^2)*q^39 + (27*t^9 + 195*t^8 + 415*t^7 + 441*t^6 + 268*t^5 + 96*t^4 + 17*t^3 + t^2)*q^38 + (17*t^9 + 150*t^8 + 350*t^7 + 389*t^6 + 247*t^5 + 91*t^4 + 17*t^3 + t^2)*q^37 + (9*t^9 + 113*t^8 + 288*t^7 + 341*t^6 + 223*t^5 + 85*t^4 + 16*t^3 + t^2)*q^36 + (5*t^9 + 83*t^8 + 239*t^7 + 298*t^6 + 204*t^5 + 80*t^4 + 16*t^3 + t^2)*q^35 + (2*t^9 + 60*t^8 + 193*t^7 + 258*t^6 + 183*t^5 + 75*t^4 + 15*t^3 + t^2)*q^34 + (t^9 + 42*t^8 + 157*t^7 + 223*t^6 + 167*t^5 + 70*t^4 + 15*t^3 + t^2)*q^33 + (28*t^8 + 123*t^7 + 191*t^6 + 148*t^5 + 65*t^4 + 14*t^3 + t^2)*q^32 + (18*t^8 + 98*t^7 + 163*t^6 + 134*t^5 + 61*t^4 + 14*t^3 + t^2)*q^31 + (11*t^8 + 74*t^7 + 137*t^6 + 118*t^5 + 56*t^4 + 13*t^3 + t^2)*q^30 + (6*t^8 + 57*t^7 + 115*t^6 + 106*t^5 + 52*t^4 + 13*t^3 + t^2)*q^29 + (3*t^8 + 41*t^7 + 95*t^6 + 92*t^5 + 48*t^4 + 12*t^3 + t^2)*q^28 + (t^8 + 30*t^7 + 78

In [10]:
series_t_q(list_of_list_of_partition_ends(w, [1, 1, 1], True))

(12*t^10 + 143*t^9 + 372*t^8 + 435*t^7 + 280*t^6 + 101*t^5 + 18*t^4 + t^3)*q^40 + (6*t^10 + 106*t^9 + 307*t^8 + 381*t^7 + 254*t^6 + 95*t^5 + 17*t^4 + t^3)*q^39 + (3*t^10 + 78*t^9 + 254*t^8 + 334*t^7 + 233*t^6 + 90*t^5 + 17*t^4 + t^3)*q^38 + (t^10 + 55*t^9 + 205*t^8 + 290*t^7 + 210*t^6 + 84*t^5 + 16*t^4 + t^3)*q^37 + (38*t^9 + 166*t^8 + 251*t^7 + 191*t^6 + 79*t^5 + 16*t^4 + t^3)*q^36 + (25*t^9 + 131*t^8 + 215*t^7 + 171*t^6 + 74*t^5 + 15*t^4 + t^3)*q^35 + (16*t^9 + 103*t^8 + 184*t^7 + 155*t^6 + 69*t^5 + 15*t^4 + t^3)*q^34 + (9*t^9 + 78*t^8 + 155*t^7 + 137*t^6 + 64*t^5 + 14*t^4 + t^3)*q^33 + (5*t^9 + 60*t^8 + 131*t^7 + 123*t^6 + 60*t^5 + 14*t^4 + t^3)*q^32 + (2*t^9 + 43*t^8 + 108*t^7 + 108*t^6 + 55*t^5 + 13*t^4 + t^3)*q^31 + (t^9 + 31*t^8 + 89*t^7 + 96*t^6 + 51*t^5 + 13*t^4 + t^3)*q^30 + (21*t^8 + 72*t^7 + 83*t^6 + 47*t^5 + 12*t^4 + t^3)*q^29 + (14*t^8 + 58*t^7 + 73*t^6 + 43*t^5 + 12*t^4 + t^3)*q^28 + (8*t^8 + 45*t^7 + 62*t^6 + 39*t^5 + 11*t^4 + t^3)*q^27 + (5*t^8 + 35*t^7 + 54*t^6 + 36*t

In [11]:
series_t_q(list_of_list_of_partition_ends(w, [1, 1, 1, 1], True))

(32*t^10 + 144*t^9 + 233*t^8 + 184*t^7 + 80*t^6 + 16*t^5 + t^4)*q^40 + (21*t^10 + 117*t^9 + 200*t^8 + 168*t^7 + 75*t^6 + 16*t^5 + t^4)*q^39 + (14*t^10 + 90*t^9 + 172*t^8 + 149*t^7 + 70*t^6 + 15*t^5 + t^4)*q^38 + (8*t^10 + 71*t^9 + 145*t^8 + 135*t^7 + 65*t^6 + 15*t^5 + t^4)*q^37 + (5*t^10 + 53*t^9 + 123*t^8 + 119*t^7 + 61*t^6 + 14*t^5 + t^4)*q^36 + (2*t^10 + 40*t^9 + 102*t^8 + 107*t^7 + 56*t^6 + 14*t^5 + t^4)*q^35 + (t^10 + 28*t^9 + 85*t^8 + 93*t^7 + 52*t^6 + 13*t^5 + t^4)*q^34 + (21*t^9 + 69*t^8 + 83*t^7 + 48*t^6 + 13*t^5 + t^4)*q^33 + (13*t^9 + 56*t^8 + 71*t^7 + 44*t^6 + 12*t^5 + t^4)*q^32 + (9*t^9 + 44*t^8 + 63*t^7 + 40*t^6 + 12*t^5 + t^4)*q^31 + (5*t^9 + 35*t^8 + 53*t^7 + 37*t^6 + 11*t^5 + t^4)*q^30 + (3*t^9 + 26*t^8 + 46*t^7 + 33*t^6 + 11*t^5 + t^4)*q^29 + (t^9 + 20*t^8 + 38*t^7 + 30*t^6 + 10*t^5 + t^4)*q^28 + (t^9 + 14*t^8 + 33*t^7 + 27*t^6 + 10*t^5 + t^4)*q^27 + (10*t^8 + 26*t^7 + 24*t^6 + 9*t^5 + t^4)*q^26 + (6*t^8 + 22*t^7 + 21*t^6 + 9*t^5 + t^4)*q^25 + (4*t^8 + 17*t^7 + 19*t^6